_A way to innovate quicker is to increase the value of a keystroke._

In this module we enhance the `ip.kernel.do_inspect` method.  The modifications to the inspector should accelerate discovery during comptational.
An author of a `pidgin` document will use __Shift+Tab__ to invoke the inspector, or they will use __The Inspector__ in [__JupyterLab__](https://github.com/jupyterlab/jupyterlab/).

    import pidgin, IPython; ip = IPython.get_ipython()

In [1]:
    if __name__ == '__main__':
        %reload_ext pidgin

In [2]:
    import pidgin, IPython, types, io, tokenize, re, ast, operator, collections, textwrap

In [3]:
__Note__: The `pidgin.inspector` module written in `pidgin` __Markdown__.  Some modifications to the inspector are `pidgin` specfic such as the inspector providing a
__Markdown__ [WYSIWYG]().
            
```ipython
>>> assert pidgin.inspector.__file__.endswith('.md.ipynb')
```

"__Note__: The `pidgin.inspector` module written in `pidgin` __Markdown__.  Some modifications to the inspector are `pidgin` specfic such as the inspector providing a\n__Markdown__ [WYSIWYG]().\n            \n```ipython\n>>> assert pidgin.inspector.__file__.endswith('.md.ipynb')\n```"

In [4]:
`tick`s and `brace`s are special `object`s in `pidgin`.  In order to discuss them properly we must assign their escaped __html__ values.

    tick, *brace = "&#96; &#123; &#125;".split()
    brace = list(map(lambda x: x*2, brace))
    
__Note__: This will be a problem later if an author uses alternative `jinja2` template delimiters.
    
    import jinja2

'`tick`s and `brace`s are special `object`s in `pidgin`.  In order to discuss them properly we must assign their escaped __html__ values.\n\n    tick, *brace = "&#96; &#123; &#125;".split()\n    brace = list(map(lambda x: x*2, brace))\n    \n__Note__: This will be a problem later if an author uses alternative `jinja2` template delimiters.\n    \n    import jinja2'

In [5]:
## Special `pidgin.inspector` indentifiers.

- __object?__ or __object??__ - one question mark returns basic info, two question marks returns the source with the `object` info.  Question marks supercede all other 
delimiters.

        question_marks = "([\S]+)\?{1}|\?{1}([\S]+)"
- __{{brace[0]}}"expression or statement"{{brace[1]}}__ - author's recieve a preview of the `jinja2` expression.

        braced = "(\{){2}([^(\?}{2})]+)\}{2}"
- __{{tick}}"expression or statement"{{tick}}__ - executes the inline code and displays a preview.  _Statements and expressions in inline code
are not rendered by `pidgin`, but intermediate representations assist in authoring `pidgin` documents._

        ticked = "`([^\?]+)`"
    
The conditions for a regular expression used to discover substrings.

        inline_code_objects = re.compile(F"{'|'.join((braced, ticked, question_marks))}")
                                        
When no `inline_code_objects` are found the `pidgin.inspector` defaults to a __Markdown__ preview.

'## Special `pidgin.inspector` indentifiers.\n\n- __object?__ or __object??__ - one question mark returns basic info, two question marks returns the source with the `object` info.  Question marks supercede all other \ndelimiters.\n\n        question_marks = "([\\S]+)\\?{1}|\\?{1}([\\S]+)"\n- __{{brace[0]}}"expression or statement"{{brace[1]}}__ - author\'s recieve a preview of the `jinja2` expression.\n\n        braced = "(\\{){2}([^(\\?}{2})]+)\\}{2}"\n- __{{tick}}"expression or statement"{{tick}}__ - executes the inline code and displays a preview.  _Statements and expressions in inline code\nare not rendered by `pidgin`, but intermediate representations assist in authoring `pidgin` documents._\n\n        ticked = "`([^\\?]+)`"\n    \nThe conditions for a regular expression used to discover substrings.\n\n        inline_code_objects = re.compile(F"{\'|\'.join((braced, ticked, question_marks))}")\n                                        \nWhen no `inline_code_objects` are found the `pidgin.inspector` defaults to a __Markdown__ preview.'

In [6]:
    def line_to_token(line, cursor, start=0):
`line_to_token` finds a token at the __cursor__ position using `inline_code_objects` regular expression.

```ipython
>>> line_to_token("this thing??", 6)
'thing??'
>>> line_to_token("questions inside ` code?? ` of stuff", 20)
'code??'
```

        token = line
    
        for m in inline_code_objects.finditer(line):
Loop over the matches discovered by `inline_code_objects`.

            start_of_segment, end = m.start(), m.end()
            if start_of_segment<=cursor<end:
                token = m.group()
                break                
Stop the loop if the segment is larger than the cursor.  Consequently, a non-computable object is returned.
            
            if cursor<start_of_segment:
                token = line[start:start_of_segment]
                break
            if m.group().strip(): start = end
            
        token = token.rstrip()
        if token.lstrip().startswith('?'): token = token.lstrip().lstrip('?')
        return token

'    def line_to_token(line, cursor, start=0):\n`line_to_token` finds a token at the __cursor__ position using `inline_code_objects` regular expression.\n\n```ipython\n>>> line_to_token("this thing??", 6)\n\'thing??\'\n>>> line_to_token("questions inside ` code?? ` of stuff", 20)\n\'code??\'\n```\n\n        token = line\n    \n        for m in inline_code_objects.finditer(line):\nLoop over the matches discovered by `inline_code_objects`.\n\n            start_of_segment, end = m.start(), m.end()\n            if start_of_segment<=cursor<end:\n                token = m.group()\n                break                \nStop the loop if the segment is larger than the cursor.  Consequently, a non-computable object is returned.\n            \n            if cursor<start_of_segment:\n                token = line[start:start_of_segment]\n                break\n            if m.group().strip(): start = end\n            \n        token = token.rstrip()\n        if token.lstrip().startswith(\'?\'): token = token.lstrip().lstrip(\'?\')\n        return token'

In [7]:
    def cursor_line_no(str, cursor_pos)->("line", "offset", "line_no"):
`cursor_line_no` takes a string and cursor position and returns the __line__ its position is in plus the __offset__ and __line_no__.  This function
provides the same utility as `IPython.utils.tokenutil.line_at_cursor` and includes the line number.

#### `cursor_line_no` tests

```ipython
>>> s = "A line\n\nfoo\nAnother"
>>> cursor_line_no(s, 10)
('foo', 8, 2)
>>> IPython.utils.tokenutil.line_at_cursor(s, 10)
('foo\n', 8)
```
---
            
            start, end = str[:cursor_pos].splitlines(True) or [''], str[cursor_pos:].splitlines(True) or ['']
            result = start.pop(-1)
            cursor_pos = cursor_pos-len(result)
            if not result.endswith('\n'): result += end[0]
            return result.rstrip(), cursor_pos, len(start)

'    def cursor_line_no(str, cursor_pos)->("line", "offset", "line_no"):\n`cursor_line_no` takes a string and cursor position and returns the __line__ its position is in plus the __offset__ and __line_no__.  This function\nprovides the same utility as `IPython.utils.tokenutil.line_at_cursor` and includes the line number.\n\n#### `cursor_line_no` tests\n\n```ipython\n>>> s = "A line\\n\\nfoo\\nAnother"\n>>> cursor_line_no(s, 10)\n(\'foo\', 8, 2)\n>>> IPython.utils.tokenutil.line_at_cursor(s, 10)\n(\'foo\\n\', 8)\n```\n---\n            \n            start, end = str[:cursor_pos].splitlines(True) or [\'\'], str[cursor_pos:].splitlines(True) or [\'\']\n            result = start.pop(-1)\n            cursor_pos = cursor_pos-len(result)\n            if not result.endswith(\'\\n\'): result += end[0]\n            return result.rstrip(), cursor_pos, len(start)'

In [8]:
    def run_nodes(ip, node, source):
`run_nodes`

        if not isinstance(node, list):
            node = [node]
        result = IPython.core.interactiveshell.ExecutionResult(
            IPython.core.interactiveshell.ExecutionInfo(source, False, False, True))
        with IPython.utils.capture.capture_output() as out:
            try:
                ip.run_ast_nodes(
                    list(map(ip.transform_ast, node)), 'inspect', 
                    interactivity='last_expr_or_assign', 
                    result=result
                ).send(None)
            except StopIteration: ...
        return dict(collections.ChainMap(
            *(object.data for object in reversed(out.outputs)),
            {
                'text/plain': repr(
                    result.error_before_exec or result.error_in_exec or out.stderr or out.stdout
                    or F">>> {source}"
                )}))

'    def run_nodes(ip, node, source):\n`run_nodes`\n\n        if not isinstance(node, list):\n            node = [node]\n        result = IPython.core.interactiveshell.ExecutionResult(\n            IPython.core.interactiveshell.ExecutionInfo(source, False, False, True))\n        with IPython.utils.capture.capture_output() as out:\n            try:\n                ip.run_ast_nodes(\n                    list(map(ip.transform_ast, node)), \'inspect\', \n                    interactivity=\'last_expr_or_assign\', \n                    result=result\n                ).send(None)\n            except StopIteration: ...\n        return dict(collections.ChainMap(\n            *(object.data for object in reversed(out.outputs)),\n            {\n                \'text/plain\': repr(\n                    result.error_before_exec or result.error_in_exec or out.stderr or out.stdout\n                    or F">>> {source}"\n                )}))'

In [9]:
    def inspect_token(shell, token, **data):
        try: 
            detail_level='??' in token
            token = IPython.utils.tokenutil.token_at_cursor(token, len(token))
            data.update(shell.object_inspect_mime(token, detail_level=detail_level))
        except KeyError: ...
        return data

In [10]:
## Using the inspector for welcome and help messaging.

A blank cell is the perfect time to present users with instructions.  `pidgin` uses the `welcome` message describe it's features.
    
    welcome =\
# Welcome to the `pidgin` inspector

The `pidgin` inspector makes it easier execute inline code and explore python `object`s.

* __?__ take priority & inspect the current `object`
* The default representation is the cell as __Markdown__ source.

---

    welcome = IPython.display.Markdown(welcome)
    
__*WIP*__

"## Using the inspector for welcome and help messaging.\n\nA blank cell is the perfect time to present users with instructions.  `pidgin` uses the `welcome` message describe it's features.\n    \n    welcome =\\\n# Welcome to the `pidgin` inspector\n\nThe `pidgin` inspector makes it easier execute inline code and explore python `object`s.\n\n* __?__ take priority & inspect the current `object`\n* The default representation is the cell as __Markdown__ source.\n\n---\n\n    welcome = IPython.display.Markdown(welcome)\n    \n__*WIP*__"

In [11]:
    _valid_brace = '{'

In [12]:
    _blocks = ast.FunctionDef, ast.ClassDef, ast.AsyncFunctionDef, ast.If, ast.For

In [13]:
    def _get_current_node(nodes, lineno, *, last_node = None): 
        while nodes and (nodes[-1].lineno-2) >= lineno: 
            last_node = nodes.pop(-1)
            
        node = nodes.pop(-1) if nodes else None
        if hasattr(node, 'body') and node.body:
            return _get_current_node(node.body, lineno, last_node=last_node)
        return node or last_node, last_node

In [14]:
    def do_inspect(kernel, code, cursor_pos, detail_level=0):
`do_inspect` is the major logic circuit for changing the inspector heavior.

"For debugging purposes", globals().update(**locals()) # tab this in twice to use it.
        
        reply = {'status' : 'ok', 'data': {}, 'metadata': {}, 'found': "This implementation always returns something" and True}
1. A welcome and help message for using `pidgin` and the `pidgin.inspector` if the __cell is blank__.
        
        if not code.strip(): 
            reply['data'].update(kernel.shell.display_formatter.format(welcome)[0])
            return reply
2. Get the __line__, __offset__, and __line number__ using `pidgin.inspector.cursor_line_no`.

        line, offset, lineno = cursor_line_no(code, cursor_pos)        
3. Access state information about the cursor position

        position_before_cursor, position_after_cursor = cursor_pos-offset-2, cursor_pos-offset-1
and when the cursor is a at the __e__nd __o__f a __l__ine.  
        
        eol = len(line) <= (1+position_after_cursor)
4. Get the actual __token__ at the cursor position.

        token = line_to_token(line, position_after_cursor+1)
## Pass the token through the logic circuits.

1. Question marks are king!
        
        if token.startswith('?') or token.endswith('?'):
            reply['data'].update(inspect_token(kernel.shell, token))
2. Are we in `tick`s?

        elif token.startswith('`') and token.endswith('`'):
        
            try:
                nodes = ast.parse(kernel.shell.input_transformer_manager.transform_cell(
                    textwrap.indent(
                    token.strip('`'),  ' '*4
                ))).body
            except: nodes = []

            reply['data'].update(run_nodes(kernel.shell, nodes, token))
3. Are we in `brace`s, or _templates_?

        elif token.startswith('{'*2): 
            reply['data'].update(
                kernel.shell.display_formatter.format(token)[0]
            )
        else:
4. Otherwise, are we in block code?

            try:
                transformed_code = kernel.shell.input_transformer_manager.transform_cell(code)
                nodes = ast.parse(transformed_code).body
            except: nodes = []
        
            node, last_node = _get_current_node(nodes, lineno)
            kernel.shell.log.error((node, last_node))
            is_str_expr = isinstance(node, ast.Expr) and isinstance(getattr(node, 'value', ast.Expr()), ast.Str)
            is_code_block = bool(line.strip() and node and eol and not is_str_expr)
            
            if is_code_block: 
Slice up the line of code that is being executed.

                lines = transformed_code.splitlines()[node.lineno-1:last_node and last_node.lineno or None]
                lines[-1] = lines[-1][:last_node and last_node.col_offset or None]
                lines[0] = lines[0][node.col_offset:]
                
                reply['data'].update(run_nodes(kernel.shell, node, '\n'.join(lines).strip()))
                
            elif line.lstrip().startswith('>>>') or line.lstrip().startswith('...') and eol and is_str_expr:
4. Return the results of a `doctest` if we are in a `doctest` block.

                with IPython.utils.capture.capture_output() as capture:
                    pidgin.doctesting.run_docstring_examples(node.value.s, verbose=True)
                reply['data'].update({'text/plain': capture.stdout})
            else: 
4. Lastly, try to inspect the object normally.  If all else show the block rendered as __Markdown__.

                reply['data'].update(
                    inspect_token(kernel.shell, token) or 
                    kernel.shell.display_formatter.format(code)[0]
                )
__Please active the __html pager__ because it makes like better.

        if not kernel.shell.enable_html_pager:  reply['data'].pop('text/html', None)
            
        return reply

'    def do_inspect(kernel, code, cursor_pos, detail_level=0):\n`do_inspect` is the major logic circuit for changing the inspector heavior.\n\n"For debugging purposes", globals().update(**locals()) # tab this in twice to use it.\n        \n        reply = {\'status\' : \'ok\', \'data\': {}, \'metadata\': {}, \'found\': "This implementation always returns something" and True}\n1. A welcome and help message for using `pidgin` and the `pidgin.inspector` if the __cell is blank__.\n        \n        if not code.strip(): \n            reply[\'data\'].update(kernel.shell.display_formatter.format(welcome)[0])\n            return reply\n2. Get the __line__, __offset__, and __line number__ using `pidgin.inspector.cursor_line_no`.\n\n        line, offset, lineno = cursor_line_no(code, cursor_pos)        \n3. Access state information about the cursor position\n\n        position_before_cursor, position_after_cursor = cursor_pos-offset-2, cursor_pos-offset-1\nand when the cursor is a at the __e__nd __o__f a __l__ine.  \n        \n        eol = len(line) <= (1+position_after_cursor)\n4. Get the actual __token__ at the cursor position.\n\n        token = line_to_token(line, position_after_cursor+1)\n## Pass the token through the logic circuits.\n\n1. Question marks are king!\n        \n        if token.startswith(\'?\') or token.endswith(\'?\'):\n            reply[\'data\'].update(inspect_token(kernel.shell, token))\n2. Are we in `tick`s?\n\n        elif token.startswith(\'`\') and token.endswith(\'`\'):\n        \n            try:\n                nodes = ast.parse(kernel.shell.input_transformer_manager.transform_cell(\n                    textwrap.indent(\n                    token.strip(\'`\'),  \' \'*4\n                ))).body\n            except: nodes = []\n\n            reply[\'data\'].update(run_nodes(kernel.shell, nodes, token))\n3. Are we in `brace`s, or _templates_?\n\n        elif token.startswith(\'{\'*2): \n            reply[\'data\'].update(\n                kernel.shell.display_formatter.format(token)[0]\n            )\n        else:\n4. Otherwise, are we in block code?\n\n            try:\n                transformed_code = kernel.shell.input_transformer_manager.transform_cell(code)\n                nodes = ast.parse(transformed_code).body\n            except: nodes = []\n        \n            node, last_node = _get_current_node(nodes, lineno)\n            kernel.shell.log.error((node, last_node))\n            is_str_expr = isinstance(node, ast.Expr) and isinstance(getattr(node, \'value\', ast.Expr()), ast.Str)\n            is_code_block = bool(line.strip() and node and eol and not is_str_expr)\n            \n            if is_code_block: \nSlice up the line of code that is being executed.\n\n                lines = transformed_code.splitlines()[node.lineno-1:last_node and last_node.lineno or None]\n                lines[-1] = lines[-1][:last_node and last_node.col_offset or None]\n                lines[0] = lines[0][node.col_offset:]\n                \n                reply[\'data\'].update(run_nodes(kernel.shell, node, \'\\n\'.join(lines).strip()))\n                \n            elif line.lstrip().startswith(\'>>>\') or line.lstrip().startswith(\'...\') and eol and is_str_expr:\n4. Return the results of a `doctest` if we are in a `doctest` block.\n\n                with IPython.utils.capture.capture_output() as capture:\n                    pidgin.doctesting.run_docstring_examples(node.value.s, verbose=True)\n                reply[\'data\'].update({\'text/plain\': capture.stdout})\n            else: \n4. Lastly, try to inspect the object normally.  If all else show the block rendered as __Markdown__.\n\n                reply[\'data\'].update(\n                    inspect_token(kernel.shell, token) or \n                    kernel.shell.display_formatter.format(code)[0]\n                )\n__Please active the __html pager__ because it makes like better.\n\n        if not kernel.shell.enable_html_pager:  reply[\'data\'].pop(

In [15]:
    def load_ipython_extension(ip): 
        if ip and hasattr(ip, 'kernel'):
Update the __ip.kernel.do_inspect__ `types.MethodType` with `do_inspect`. 

            ip.kernel.do_inspect = types.MethodType(do_inspect, ip.kernel)
            ip.enable_html_pager = True
    def unload_ipython_extension(ip): 
        if ip and hasattr(ip, 'kernel'):
            ip.kernel.do_inspect = types.MethodType(type(ip.kernel).do_inspect, ip.kernel)
    if __name__ == '__main__': load_ipython_extension(IPython.get_ipython())  

"    def load_ipython_extension(ip): \n        if ip and hasattr(ip, 'kernel'):\nUpdate the __ip.kernel.do_inspect__ `types.MethodType` with `do_inspect`. \n\n            ip.kernel.do_inspect = types.MethodType(do_inspect, ip.kernel)\n            ip.enable_html_pager = True\n    def unload_ipython_extension(ip): \n        if ip and hasattr(ip, 'kernel'):\n            ip.kernel.do_inspect = types.MethodType(type(ip.kernel).do_inspect, ip.kernel)\n    if __name__ == '__main__': load_ipython_extension(IPython.get_ipython())  "

This module was adapted from a blog `deathbeds` blog post `deathbeds.__The_inspector_should_be_a_teacher`.